In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# read in data
data_offline= pd.read_csv('ccf_offline.csv')
data_online = pd.read_csv('ccf_online.csv')

## get online merchant table function

In [3]:

def get_online_merchant_table(data_online):
    #get num_coupon for each user
    
    a=data_online[['Merchant_id','Coupon_id']]
    mask = data_online['Coupon_id']!='null'
    num_coupon = a[mask].groupby(['Merchant_id']).count().reset_index()
    num_coupon.columns = ['Merchant_id','Tot_coupon_sent']
    data_online1 = pd.merge(data_online, num_coupon, on = 'Merchant_id', how='left')

    #get num coupon used for each user
    mask2 = (data_online1['Date']!= 'null') & (data_online1['Coupon_id']!= 'null')
    b=data_online1[mask2].groupby(['Merchant_id'])['Action'].count().reset_index()
    b.columns = ['Merchant_id','Num_coupon_used']
    data_online2 = pd.merge(data_online1, b, on='Merchant_id', how='left')

    #get total number of coupons type for each user
    l= data_online2[['Merchant_id','Coupon_id','Action']].groupby(['Merchant_id','Coupon_id']).count().reset_index()
    mask3 = l['Coupon_id']!='null'
    l2=l[mask3]

    def num_type(s):
        if s>0:
            return 1

    l2['num_type']=l2['Coupon_id'].apply(num_type)
    l3 =l2.groupby('Merchant_id')['num_type'].count()
    l4=pd.DataFrame(l3).reset_index()
    data_online3= pd.merge(data_online2, l4, on='Merchant_id',how='left')

    #generate merchant table
    no_userid= data_online3[['Merchant_id','Tot_coupon_sent','Num_coupon_used','num_type']]
    merchant_online_table = no_userid.drop_duplicates()
    return merchant_online_table

In [14]:
from datetime import datetime 
startTime= datetime.now() 
me=get_online_merchant_table(data_online)
timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:32.275884


In [15]:
me.head()

,Merchant_id,Tot_coupon_sent,Num_coupon_used,num_type
0,18907,52552.0,10927.0,152.0
1,34805,3334.0,330.0,90.0
10,38810,117.0,5.0,7.0
16,37005,498.0,128.0,9.0
17,14305,34924.0,9221.0,236.0


## get offline merchant table function

In [5]:
def get_offline_merchant_table(data_offline):
    a=data_offline[['Merchant_id','Coupon_id']]
    mask = data_offline['Coupon_id']!='null'
    num_coupon = a[mask].groupby(['Merchant_id']).count().reset_index()
    num_coupon.columns = ['Merchant_id','Tot_coupon_sent']
    data_offline1 = pd.merge(data_offline, num_coupon, on = 'Merchant_id',how='left')
    mask2 = (data_offline1['Date']!= 'null') & (data_offline1['Coupon_id']!= 'null')
    b=data_offline1[mask2].groupby(['Merchant_id'])['Coupon_id'].count().reset_index()
    b.columns = ['Merchant_id','Num_coupon_used']
    data_offline2 = pd.merge(data_offline1, b, on='Merchant_id',how='left')
    l= data_offline2[['Merchant_id','Coupon_id']].groupby(['Merchant_id','Coupon_id']).count().reset_index()
    mask3 = l['Coupon_id']!='null'
    l2=l[mask3]
    
    def num_type(s):
        if s>0:
            return 1
        
    l2['num_type']=l2['Coupon_id'].apply(num_type)
    l3 =l2.groupby('Merchant_id')['num_type'].count()
    l4=pd.DataFrame(l3).reset_index()
    data_offline3= pd.merge(data_offline2, l4, on='Merchant_id',how='left')
    null_mask = data_offline3['Distance']!='null'

    def to_float(s):
        return float(s)
    data_offline4 =data_offline3[null_mask]
    data_offline4['Distance']=data_offline4['Distance'].map(to_float)
    data_offline5 = pd.DataFrame(data_offline4.groupby('Merchant_id')['Distance'].mean()).reset_index()
    data_offline6 = pd.merge(data_offline3, data_offline5,on='Merchant_id',how='left')
    data_offline6=data_offline6.rename(columns = {'Distance_y':'Avg_Distance'})
    
    no_others= data_offline6[['Merchant_id','Tot_coupon_sent','Num_coupon_used','num_type','Avg_Distance']]
    merchant_offline_table = no_others.drop_duplicates()
    return merchant_offline_table

In [6]:
from datetime import datetime 
startTime= datetime.now() 
get_offline_merchant_table(data_offline)
timeElapsed=datetime.now()-startTime 
print('Time elpased (hour:min:second.ms) {}'.format(timeElapsed))

Time elpased (hour:min:second.ms) 0:00:05.486122
